In [ ]:
# Calculate the correlations between the Law Dome ice core proxy records and SST values from ERSSTv5 using a multivariate linear regression
#
# This investigates increase in correlation or spatial skill by the inclusion of more species, and by smoothing the datasets

In [1]:
from dask.distributed import Client
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 5
Total threads: 20,Total memory: 30.96 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:37751,Workers: 5
Dashboard: http://127.0.0.1:8787/status,Total threads: 20
Started: Just now,Total memory: 30.96 GiB
Comm: tcp://127.0.0.1:36377,Total threads: 4
Dashboard: http://127.0.0.1:45493/status,Memory: 6.19 GiB
Nanny: tcp://127.0.0.1:44849,


In [2]:
import numpy as np
import math
import xarray as xr


import matplotlib.pyplot as plt
import matplotlib.font_manager

matplotlib.font_manager.findSystemFonts(fontpaths=None, fontext='ttf')
import cartopy.crs as ccrs

import datetime
today=datetime.date.today()

from arch.bootstrap import CircularBlockBootstrap
from scipy.stats import pearsonr
from scipy.ndimage import gaussian_filter
import pandas as pd

import matplotlib.ticker as ticker
plt.rcParams['font.family'] = 'Arial'
#plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = 'Arial'

In [12]:
#read in training data (ie, instrumental period.)
#read the ice core data
#this file is can be downloaded from the AADC here https://doi.org/10.26179/5zm0-v192 put it in the data directory
df=pd.read_csv('data/DSS_2k_winter_centred.csv')
df.rename(columns={'Year (CE)':'year','DJFMAM (μEq/L)':'djfmam','JJASON (μEq/L)':'jjason','Accumulation rate (IE m/y)':'accum'}, inplace=True)
df.set_index(['year'], inplace=True)
df=df.iloc[::-1]
icecoredata = xr.Dataset.from_dataframe(df)
icecoredata

<xarray.Dataset> Size: 195kB
Dimensions:             (year: 2028)
Coordinates:
  * year                (year) int64 16kB -11 -10 -9 -8 ... 2013 2014 2015 2016
Data variables:
    Year BP1950         (year) int64 16kB 1961 1960 1959 1958 ... -64 -65 -66
    Sodium (μEq/L)      (year) float64 16kB 2.925 2.638 4.173 ... 3.275 4.326
    Chloride (μEq/L)    (year) float64 16kB 3.687 3.249 5.183 ... 3.847 4.994
    Magnesium (μEq/L)   (year) float64 16kB 0.46 0.455 0.73 ... 0.629 0.844
    Sulphate (μEq/L)    (year) float64 16kB 0.589 0.499 0.723 ... 0.724 0.795
    (d18O)              (year) float64 16kB -22.1 -22.08 ... -22.75 -22.51
    layer_thickness(m)  (year) float64 16kB 0.18 0.178 0.281 ... 0.692 0.7334
    accum               (year) float64 16kB 0.6621 0.6544 ... 0.6943 0.7346
    djfmam              (year) float64 16kB nan nan 0.373 ... -0.067 0.103 0.154
    jjason              (year) float64 16kB nan 0.543 0.757 ... 0.528 0.663 0.83
    DJFM (μEq/L)        (year) float64 16kB nan nan 0.1552 ... 0.04611 0.3387

In [13]:
#set up the ice core data with xarray
calibration_years=np.arange(1854,2017) #

#fill the gaps by interpolation
sst_proxy_data_tmp=icecoredata[['year','djfmam','jjason','accum']].interpolate_na(dim='year',max_gap=4,method="linear", fill_value="extrapolate")
sst_proxy_data=sst_proxy_data_tmp.to_dataarray()

In [5]:
#ersst is the reconstruction target, calulate the annual average ssta from the monthly data
ersst=xr.open_mfdataset('data/ersst_v5/*.nc',parallel=True) #or change to wherever you have the ersstv5 dataset stored
annAv_ersst=ersst['ssta'].groupby('time.year').mean(skipna=True)
annAv_ersst=annAv_ersst.fillna(0)

annAv_ersst=annAv_ersst.chunk(dict(year=-1))

annAv_ersst=annAv_ersst.isel(lev=0)
annAv_ersst.compute()

#calculate a few things for convenience
years=len(annAv_ersst.year)
nlats=len(annAv_ersst.lat)
nlons=len(annAv_ersst.lon)

h_scale=0.25

lat = annAv_ersst['lat'].values
lon = annAv_ersst['lon'].values
sst = annAv_ersst.values

In [45]:
#create data structures for the training data
data_train = sst_proxy_data.sel(dict(year=slice(1854,2017),variable=['djfmam','jjason','accum'])).transpose()
data_full=sst_proxy_data.sel(dict(year=slice(-9,2017),variable=['djfmam','jjason','accum'])).transpose()


('year', 'variable')

In [55]:

data_train_sss=data_train.sel(dict(variable='djfmam'))
data_train_wss=data_train.sel(dict(variable='jjason'))
data_train_sss_wss=data_train.sel(dict(variable=['djfmam','jjason']))
data_train_acc=data_train.sel(dict(variable='accum'))

data_full_sss=data_full.sel(dict(variable='djfmam'))
data_full_wss=data_full.sel(dict(variable='jjason'))
data_full_sss_wss=data_full.sel(dict(variable=['djfmam','jjason']))
data_full_acc=data_full.sel(dict(variable='accum'))

In [58]:
def pearson(x,y):
    r,p=pearsonr(x,y)
    if math.isnan(r):
        r=0.0
    return r

In [59]:
def corr_only(train, ttarget):
    corr_coeff = pearson(ttarget, train)[0]
    return corr_coeff


In [46]:
from sklearn.linear_model import LinearRegression

def new_linreg(train, ttarget,full_data):
    # Wrapper around scipy linregress to use in apply_ufunc
    reg = LinearRegression()
   # print(target)

   # ttarget=gaussian_filter(ttarget,sigma=sigma)

    model = reg.fit(train, ttarget)

    ypred = model.predict(train)
    corr_coeff=pearsonr(ttarget,ypred)[0]

    ssta_recotmp=reg.predict(full_data)

    return (ypred,corr_coeff,ssta_recotmp)


In [31]:
def bootstraps(ttarget, ypred,corr_coeff):

    bs = CircularBlockBootstrap(13, years,x=ttarget,y=ypred) #checked autocorrelation length of the data, 6 is consistant with block length chosen by Mudelsee code

    if np.isnan(corr_coeff):
        conf_interval=np.zeros(2)
    else:
        conf_interval=bs.conf_int(pearson,reps=2000,method='bca').flatten()

    return conf_interval

In [ ]:
#calculate correlation, prediction and full reconstruction
ssta_pred,corrs,ssta_reco=xr.apply_ufunc(new_linreg,
                    data_train,
                    annAv_ersst,
                    data_full,
                    join='inner',
                    input_core_dims=[['year','variable'],['year'],['year','variable']],
                    output_core_dims=[['year'],[],['year']],
                    output_dtypes=[np.float32,np.float32,np.float32],
                    vectorize=True,
                    dask='parallelized',
                    )

In [ ]:
ssta_pred = ssta_pred.persist()
corrs = corrs.persist()
ssta_reco = ssta_reco.persist()

In [ ]:

ssta_pred_sss, corrs_sss, ssta_reco_sss = xr.apply_ufunc(new_linreg,
                                                         data_train_sss,
                                                         annAv_ersst,
                                                         data_full_sss,
                                                         input_core_dims=[['year', 'variable'], ['year'], ['year', 'variable']],
                                                         output_core_dims=[['year'], [], ['year']],
                                                         join='inner',
                                                         output_dtypes=[np.float32, np.float32, np.float32],
                                                         vectorize=True,
                                                         dask='parallelized',
                                                         )

ssta_pred_sss_wss, corrs_sss_wss, ssta_reco_sss_wss = xr.apply_ufunc(new_linreg,
                                                                     data_train_sss_wss,
                                                                     annAv_ersst,
                                                                     data_full_sss_wss,
                                                                     input_core_dims=[['year', 'variable'], ['year'],
                                                                                      ['year', 'variable']],
                                                                     join='inner',
                                                                     output_core_dims=[['year'], [], ['year']],
                                                                     output_dtypes=[np.float32, np.float32, np.float32],
                                                                     vectorize=True,
                                                                     dask='parallelized',
                                                                     )



In [ ]:
ssta_pred_sss.persist()
corrs_sss.persist()
ssta_reco_sss.persist()

ssta_pred_sss_wss.persist()
corrs_sss_wss.persist()
ssta_reco_sss_wss.persist()

In [ ]:
conf_int = xr.apply_ufunc(bootstraps,
                          annAv_ersst,
                          ssta_pred,
                          corrs,
                          input_core_dims=[['year'], ['year'], []],
                          output_sizes={'bound': 2},
                          output_core_dims=[['bound']],
                          output_dtypes=[np.float32],
                          vectorize=True,
                          dask='parallelized',
                          )
conf_int_sss = xr.apply_ufunc(bootstraps,
                              annAv_ersst,
                              ssta_pred_sss,
                              corrs_sss,
                              input_core_dims=[['year'], ['year'], []],
                              output_sizes={'bound': 2},
                              output_core_dims=[['bound']],
                              output_dtypes=[np.float32],
                              vectorize=True,
                              dask='parallelized',
                              )
conf_int_sss_wss = xr.apply_ufunc(bootstraps,
                                  annAv_ersst,
                                  ssta_pred_sss_wss,
                                  corrs_sss_wss,
                                  input_core_dims=[['year'], ['year'], []],
                                  output_sizes={'bound': 2},
                                  output_core_dims=[['bound']],
                                  output_dtypes=[np.float32],
                                  vectorize=True,
                                  dask='parallelized',
                                  )

In [ ]:
conf_int.persist()
conf_int_sss.persist()
conf_int_sss_wss.persist()

In [ ]:
#write outputs to netcdf files
ds = conf_int.transpose('bound', 'lat', 'lon').to_dataset(name='conf_interval')
ds.to_netcdf("outputs/unsmoothed/ssta_cis.nc", mode='w')

ds = conf_int_sss.transpose('bound', 'lat', 'lon').to_dataset(name='conf_interval')
ds.to_netcdf("outputs/unsmoothed/ssta_sss_cis.nc", mode='w')

ds = conf_int_sss_wss.transpose('bound', 'lat', 'lon').to_dataset(name='conf_interval')
ds.to_netcdf("outputs/unsmoothed/ssta_sss_wss_cis.nc", mode='w')

ds = ssta_reco.transpose('year', 'lat', 'lon').to_dataset(name='ssta_reconstructed')
ds.to_netcdf("outputs/unsmoothed/ssta_reco.nc", mode='w')

ds = ssta_reco_sss.transpose('year', 'lat', 'lon').to_dataset(name='ssta_reconstructed')
ds.to_netcdf("outputs/unsmoothed/ssta_reco_sss.nc", mode='w')

ds = ssta_reco_sss_wss.transpose('year', 'lat', 'lon').to_dataset(name='ssta_reconstructed')
ds.to_netcdf("utputs/unsmoothed/ssta_reco_sss_wss.nc", mode='w')

ds = ssta_pred.transpose('year', 'lat', 'lon').to_dataset(name='ssta')
ds.to_netcdf("outputs/unsmoothed/ssta_pred.nc", mode='w')

ds = ssta_pred_sss.transpose('year', 'lat', 'lon').to_dataset(name='ssta')
ds.to_netcdf("outputs/unsmoothed/ssta_pred_sss.nc", mode='w')

ds = ssta_pred_sss_wss.transpose('year', 'lat', 'lon').to_dataset(name='ssta')
ds.to_netcdf("outputs/unsmoothed/ssta_pred_sss_wss.nc", mode='w')

ds = corrs.to_dataset(name='correlation_coefficient')
ds.to_netcdf("outputs/unsmoothed/corr_coeff.nc", mode='w')

ds = corrs_sss.to_dataset(name='correlation_coefficient')
ds.to_netcdf("utputs/unsmoothed/corr_coeff_sss.nc", mode='w')

ds = corrs_sss_wss.to_dataset(name='correlation_coefficient')
ds.to_netcdf("outputs/unsmoothed/corr_coeff_sss_wss.nc", mode='w')

In [64]:
# caluclate the separate correlation coefficients for winter sea salts and accumulation

corrs_wss=xr.corr(data_train_wss.squeeze('variable'),annAv_ersst,dim='year')
corrs_wss.persist()

corrs_acc=xr.corr(data_train_acc.squeeze('varialbe'),annAv_ersst,dim='year')
corrs_acc.persist()


<xarray.DataArray (lat: 89, lon: 180)> Size: 128kB
dask.array<truediv, shape=(89, 180), dtype=float64, chunksize=(89, 180), chunktype=numpy.ndarray>
Coordinates:
    variable  <U6 24B 'djfmam'
  * lat       (lat) float64 712B -88.0 -86.0 -84.0 -82.0 ... 82.0 84.0 86.0 88.0
    lev       float64 8B 0.0
  * lon       (lon) float64 1kB 0.0 2.0 4.0 6.0 8.0 ... 352.0 354.0 356.0 358.0

In [61]:

ds = corrs_wss.to_dataset(name='correlation_coefficient')
ds.to_netcdf("utputs/unsmoothed/corr_coeff_wss.nc", mode='w')

ds = corrs_accumto_dataset(name='correlation_coefficient')
ds.to_netcdf("outputs/unsmoothed/corr_coeff_accum.nc", mode='w')

KeyError: 'variable'